<a href="https://colab.research.google.com/github/syedareehaquasar/WTEF/blob/master/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorators



In [ ]:
# recap 
def decorator_fn(func):
    def decorated(a, b):
        if b == 0:
            print('Integer division by 0 not permitted')
            return 0
        return func(a, b)
    return decorated

@decorator_fn
def divide1(x, y):
    return x // y

print(divide1(6, 2))
print(divide1(6, 0))

3
Integer division by 0 not permitted
0


hello() decorator wraps name()

![](https://d33wubrfki0l68.cloudfront.net/12c8a296cc396d418b5407a4a4c6f9fd7d85f597/e8a54/wp-content/uploads/2018/06/python-decorator.png)

![](https://d585tldpucybw.cloudfront.net/sfimages/default-source/blogs/2019/2019-12/decoratorConcept.png)

Why decorators are useful?

1. By making a logger function:

(make sure the wrapper function (logging_wrapper) returns the return value of the decorated function (func))

In [ ]:
def logging_decorator(func):
  def logging_wrapper(*args, **kwargs):
    print(f'Running {func.__name__}() ...')
    result = func(*args, **kwargs)
    print(f'Result: {result}')
    print(f'{func.__name__}() exited')
    return result
  return logging_wrapper

In [ ]:
@logging_decorator
def sum(x, y):
  return (x + y)
  
@logging_decorator
def sub(x, y):
  return (x - y)

added = sum(2, 1)
subtracted = sub(2, 1)

Running sum() ...
Result: 3
sum() exited
Running sub() ...
Result: 1
sub() exited



  -  see result of functions on every step
  - finding errors (by making something like a call stack)

In [ ]:
@logging_decorator
def moon():
  # return "o" 
  return "o" + 2

@logging_decorator
def stars():
  return "*"

@logging_decorator
def night_sky():
  return stars() + moon() + stars()

night_sky()

Running night_sky() ...
Running stars() ...
Result: *
stars() exited
Running moon() ...


TypeError: ignored

2. Validation and runtime checks

  Check that the result is whats its supposed to be

In [ ]:
def validate_summary(func):
    def wrapper(*args, **kwargs):
        data = func(*args, **kwargs)
        if len(data["summary"]) > 80:
            raise ValueError("Summary too long")
        return data
    return wrapper

@validate_summary
def queryA(params):
    # ...

@validate_summary
def queryB(params):
    # ...

@validate_summary
def queryC(params):
    # ...

3.  If there is any behaviour that is common to more than one function, you probably need to make a decorator.

4. Reusing code

  Consider working with a flakey API. You make requests to something that return JSON over HTTP, and it works correctly 99.9% of the time. But… a small fraction of all requests will cause the server to return an internal error, and you need to retry the request. 
  
  In that case, you’d implement some retry logic:

In [ ]:
# ...
response = None
while True:
    response = make_api_call()
    if response.status_code == 500 and tries < MAX_TRIES:
        tries += 1
        continue
    break
# ...

Now imagine you have dozens of functions like make_api_call(), and they are called all over the codebase. Are you going to implement that while loop everywhere?

In [ ]:
def retry(func):
    def retried_func(*args, **kwargs):
        MAX_TRIES = 3
        tries = 0
        while True:
            response = func(*args, **kwargs)
            if response.status_code == 500 and tries < MAX_TRIES:
                tries += 1
                continue
            break
        return response
    return retried_func

@retry
def make_api_call():
    # ....

5. Creating frameworks (abstraction yay!)

  Example: Flask! 
  That `route` method returns a decorator that is applied to the handler function. All other complexity is completely hidden.

In [ ]:
# For a RESTful todo-list API.
@app.route("/tasks/", methods=["GET"])
def get_all_tasks():
    tasks = app.store.get_all_tasks()
    return make_response(json.dumps(tasks), 200)

@app.route("/tasks/", methods=["POST"])
def create_task():
    payload = request.get_json(force=True)
    task_id = app.store.create_task(
        summary = payload["summary"],
        description = payload["description"],
    )
    task_info = {"id": task_id}
    return make_response(json.dumps(task_info), 201)

@app.route("/tasks/<int:task_id>/")
def task_details(task_id):
    task_info = app.store.task_details(task_id)
    if task_info is None:
        return make_response("", 404)
    return json.dumps(task_info)


Using decorators:



1. Decorators on classes

  i) decorate the methods of a class

    The @property decorator is used to customize getters and setters for class attributes.
    Properties are accessed as attributes without parentheses.

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter 
    def radius(self, value):
        """Set radius value"""
        self._radius = value

circle = Circle(5)
print(circle.radius)
circle.radius = 4
print(circle.radius)

4

  ii) decorate the whole class

  Decorating a class does not decorate its methods.

In [ ]:
@logging_decorator
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            # do something
            for _ in range(num_times):
                pass
        return 0

tw = TimeWaster(1000)
tw.waste_time(2)
print(type(TimeWaster))
print(type(tw))

Running TimeWaster() ...
Result: <__main__.TimeWaster object at 0x7f6dfcfb5198>
TimeWaster() exited
<class 'function'>
<class '__main__.TimeWaster'>


2. Classes as decorators

  - decorate functions

A callable object is an object which can be used and behaves like a function but might not be a function. It is possible to define classes in a way that the instances will be callable objects. The __call__ method is called, if the instance is called "like a function", i.e. using brackets.

In [ ]:
# decorator class, a callable object
class Counter:
    def __init__(self, func):
        print("Counter init")
        self.func = func
        self._num_calls = 0

    def __call__(self, *args, **kwargs):
        self._num_calls += 1
        print(f"Call {self._num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

    @property # GETTER
    def num_calls(self):
        return self._num_calls

    def do(self):
        return "a"

    def do3(self):
      return "C"

# callabe object? 
# counter = Counter()
# counter()

In [ ]:
@Counter
def say_whee():
    print("Whee!")

say_whee()
say_whee()
say_whee()

print(say_whee.num_calls)
print(type(say_whee))

Counter init
Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!
Call 3 of 'say_whee'
Whee!
3
<class '__main__.Counter'>


  - decorate classes

    Decorating a class does not decorate its methods.

In [ ]:
@Counter
class Do:
    def __init__(self):
      print("Do init")
      self.doer = "harry"
    def do(self):
      return "A"
    def do2(self):
      return "B"

does = Do()
print(does.doer)
print(does.do2())
print(does.do())
# print(does.do3())
does2 = Do()
print(type(Do))
print(type(does))

Counter init
Call 1 of 'Do'
Do init
harry
B
A
Call 2 of 'Do'
Do init
<class '__main__.Counter'>
<class '__main__.Do'>


Counter decorator decorates the __init__() constructor and not the Do class, in a sense. 


When we call Do(), it makes a Counter object then because we called it () it runs __call__() which returns func() which here is Do() instance.

### Refernces

- https://realpython.com/primer-on-python-decorators/
- https://www.oreilly.com/content/5-reasons-you-need-to-learn-to-write-python-decorators/
- https://www.python-course.eu/python3_decorators.php